![alt text](../../pythonexposed-high-resolution-logo-black.jpg "Optionele titel")

### Named Tupels - Aanpassen en Uitbreiden

In [3]:
from collections import namedtuple

In [4]:
Point2D = namedtuple('Point2D', 'x y')

De objecten gegenereerd door `namedtuple` gegenereerde klassen zijn **onveranderlijk**.
Het volgende zal dus niet werken:

In [5]:
origin = Point2D(10,0)

In [6]:
origin.x = 0

AttributeError: can't set attribute

Wat als we de waarde van één van de coördinaten van onze `origin` variabele willen "wijzigen"?  
Dit is net zoals bij strings: we moeten een nieuwe versie van de tuple creëren en deze toewijzen aan hetzelfde label.

Stel dat we de x-coördinaat van onze origin willen veranderen naar iets anders, maar de y-coördinaat willen behouden zoals die was.

We zouden dit als volgt kunnen doen:

In [7]:
origin = Point2D(0, origin.y)

In [8]:
origin

Point2D(x=0, y=0)

Natuurlijk kan dit behoorlijk onhandig worden wanneer we een groter aantal eigenschappen hebben en we slechts één item hoeven te wijzigen:

In [9]:
Stock = namedtuple('Stock', 'symbol year month day open high low close')

In [10]:
djia = Stock('DJIA', 2018, 1, 25, 26_313, 26_458, 26_260, 26_393)

Om bijvoorbeeld de `close` eigenschap bij te werken, zouden we kunnen schrijven:

In [11]:
djia = Stock(djia.symbol, djia.year, djia.month, djia.day, 
                  djia.open, djia.high, djia.low, 26_394)

Dat was bijzonder onhandig...
We kunnen hier iets slimmer mee omgaan en tuple-unpacking en argument-unpacking als volgt gebruiken:

In [12]:
*values, _ = djia

We gaven geen aandacht aan de `close` prijs aangezien we deze vervangen, vandaar de underscore variabelenaam.
Nu hebben we al de rest in een list:

In [13]:
values

['DJIA', 2018, 1, 25, 26313, 26458, 26260]

En nu gaan we de `*` opnieuw gebruiken, maar deze keer om de lijst uit te pakken in afzonderlijke argumenten wanneer we de `Stock` initializer aanroepen:

In [14]:
djia = Stock(*values, 26_393)

In [15]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

Dit is veel beter dan onze eerste poging!
Maar deze aanpak werkt niet altijd: wat gebeurt er als we een waarde in het midden willen veranderen? Of twee waarden?

Dit zal niet werken:
*first, month, *last = djia

Misschien kunnen slicing en unpacking hier wel werken...

In [16]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

We proberen **slicing**:

In [17]:
djia[:3]

('DJIA', 2018, 1)

In [18]:
djia[:3] + (26,) + djia[4:]

('DJIA', 2018, 1, 26, 26313, 26458, 26260, 26393)

Dus nu kunnen we dit gebruiken om een nieuwe StockPrice-instantie te maken:

In [19]:
djia2 = Stock(*(djia[:3] + (26,) + djia[4:]))

In [18]:
djia2

Stock(symbol='DJIA', year=2018, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

Dit werkt, maar is vrij omslachtig...
En het wordt erger - stel dat we het jaar en de dag willen wijzigen met deze aanpak:

In [20]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [21]:
values = djia[0:1] + (2019,) + djia[2:3] + (26,) + djia[4:]

In [22]:
values

('DJIA', 2019, 1, 26, 26313, 26458, 26260, 26393)

In [25]:
djia3 = Stock(*values)

In [26]:
djia3

Stock(symbol='DJIA', year=2019, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

Dit wordt echt te complex.
Gelukkig is er een betere manier!

De implementatie van namedtuple biedt ook een andere methode genaamd _replace die alleen keyword-argumenten accepteert. Deze methode maakt een kopie van de huidige tuple en vervangt eigenschapswaarden op basis van de doorgegeven keyword-argumenten.

In [27]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [28]:
id(djia)

139744146363600

In [29]:
djia5 = djia._replace(year=2019, day=26)

In [30]:
djia5

Stock(symbol='DJIA', year=2019, month=1, day=26, open=26313, high=26458, low=26260, close=26393)

In [31]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [32]:
id(djia5)

139744146369424

Dit werkt veel overzichtelijker...

#### Uitbreiden van Named Tuples

Soms willen we één of meer eigenschappen toevoegen aan een bestaande class zonder de code van de aangepaste class zelf te wijzigen.
Het gebruik van inheritance is een manier om dit aan te pakken.  Je zou dus geneigd kunnen zijn dit ook met named tuples te doen, maar dat is niet eenvoudig, en er is een betere manier om dit te doen als je alleen extra datavelden wilt toevoegen.

Stel dat we een Point klasse hebben die bedoeld is voor 2D-problemen:

In [32]:
Point2D = namedtuple('Point2D', 'x y')

We kunnen gemakkelijk een 3D-puntklasse maken als volgt:

In [33]:
Point3D = namedtuple('Point3D', 'x y z')

Maar als onze genoemde tuple veel velden heeft, zoals onze `Stock` genoemde tuple, dan is dat iets lastiger:

In [34]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

Stel dat we een nieuwe klasse willen maken, laten we zeggen `StockExt`, dan zou dat wat moeite kosten:

In [35]:
StockExt = namedtuple('StockExt', 
                      '''symbol year month day open high low 
                      close previous_close''')

In plaats daarvan kunnen we gebruikmaken van de `_fields` eigenschap:

In [36]:
Stock._fields

('symbol', 'year', 'month', 'day', 'open', 'high', 'low', 'close')

Onthoud dat de `namedtuple` initializer een lijst of tuple kan verwerken met daarin de veldnamen. Bijvoorbeeld, degene die we zojuist hebben opgehaald uit `_fields`.
Nu hoeven we alleen maar een nieuwe tuple te creëren die die velden bevat samen met welke extra's we willen:

In [33]:
new_fields = Stock._fields + ('previous_close',)

In [38]:
new_fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

En nu kunnen we onze nieuwe named tuple op deze manier maken:

In [34]:
StockExt = namedtuple('StockExt', Stock._fields + ('previous_close',))

In [40]:
StockExt._fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

Je zou dit ook kunnen doen met strings:

In [35]:
' '.join(Stock._fields) + ' previous_close'

'symbol year month day open high low close previous_close'

In [36]:
StockExt = namedtuple('StockExt', 
                      ' '.join(Stock._fields) + ' previous_close')

In [37]:
StockExt._fields

('symbol',
 'year',
 'month',
 'day',
 'open',
 'high',
 'low',
 'close',
 'previous_close')

Nu, met deze nieuw uitgebreide class, willen we misschien een van de "oude" named tuple instanties (`djia`) nemen en de uitgebreide versie ervan maken met behulp van de `StockExt` class.
Dit is ook vrij eenvoudig te doen, aangezien named tuples tuples zijn, en dus kunnen worden unpacked in de argumenten van een functieoproep.

In [44]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [45]:
djia_ext = StockExt(*djia, 25_000)

In [46]:
djia_ext

StockExt(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393, previous_close=25000)

of, we kunnen de `_make` methode gebruiken:

In [47]:
djia_ext = StockExt._make(djia + (25_000, ))

In [48]:
djia_ext

StockExt(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393, previous_close=25000)